In [1]:
%run config.ipynb

In [2]:
from numpy import newaxis
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Convolution1D, MaxPooling1D, Flatten,  Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from datetime import datetime

Using TensorFlow backend.


In [3]:
# Connect to Cortex
cortex = Cortex.client()

In [4]:
# Our stock symbols
symbols = ['fb', 'xlf', 'aapl']

In [5]:
s = symbols[0]
print('Loading stock prices for %s' % s)
ds = cortex.dataset('demo/stock-prices-%s' % s)
c = ds.contract('stock-prices')
df = c.load_feature_file(tag='prices_%s' % s)
c.discover.data_dictionary(df)

Loading stock prices for fb


,Attribute,Type,% Nulls,Count,Unique,Observations,Knowledge
0,close,float64,0.0,1259,1201,max=2.3869951845748765 | min=-1.7078579600615227 | mean=0.0,


In [6]:
look_back = 30
holdout_date = pd.to_datetime('2018-01-01')

train_df = df.loc[df.index < holdout_date]

# Convert our target variable to the right output format (2d array)
timeseries = np.asarray(train_df.close)
timeseries = np.atleast_2d(timeseries)
if timeseries.shape[0] == 1:
    timeseries = timeseries.T
    
timeseries.shape

(1107, 1)

In [7]:
# Setup training instances
X = np.atleast_3d(np.array([timeseries[start:start + look_back] for start in range(0, timeseries.shape[0] - look_back)]))
y = timeseries[look_back:]

print(X.shape)
print(X[0])
print(y.shape)
print(y[0:4])

(1077, 30, 1)
[[-1.66395382]
 [-1.6702905 ]
 [-1.69747033]
 [-1.70582117]
 [-1.70785796]
 [-1.69608984]
 [-1.67956921]
 [-1.66599061]
 [-1.6680274 ]
 [-1.66282227]
 [-1.61756027]
 [-1.59968178]
 [-1.63815448]
 [-1.61760553]
 [-1.60103964]
 [-1.60072281]
 [-1.58768735]
 [-1.58972414]
 [-1.56980886]
 [-1.54061487]
 [-1.53857808]
 [-1.54853572]
 [-1.51594708]
 [-1.52251007]
 [-1.53246771]
 [-1.57320351]
 [-1.51526815]
 [-1.51164719]
 [-1.49467394]
 [-1.46050113]]
(1077, 1)
[[-1.46729043]
 [-1.43877537]
 [-1.41591806]
 [-1.39487123]]


In [8]:
model = Sequential()
model.add(LSTM(input_shape = (1,), input_dim=1, output_dim=6, return_sequences=True))
model.add(LSTM(input_shape = (1,), input_dim=1, output_dim=6, return_sequences=False))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss="mse", optimizer="rmsprop")

In [ ]:
from keras_tqdm import TQDMNotebookCallback

callbacks = []
# callbacks.append(TQDMNotebookCallback(leave_inner=False, leave_outer=True))

model.fit(X, 
          y, 
          epochs=1000, 
          batch_size=80, shuffle=False, callbacks=callbacks)

In [10]:
# Save model
model.save(filepath='model_stock_prices_%s.h5' % s, overwrite=True)